**Import libraries**

In [37]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, roc_curve, roc_auc_score
import nltk
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.datasets import make_classification
from sklearn.ensemble import GradientBoostingClassifier

import sklearn
from imblearn.combine import SMOTETomek
from imblearn.under_sampling import NearMiss
from imblearn.over_sampling import RandomOverSampler
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn import svm
import warnings
warnings.simplefilter("ignore")

**Load the dataset**

In [ ]:
# from google.colab import drive
# drive.mount('/gdrive', force_remount=True)

In [8]:
# Load dataset
def load_data():
    data =pd.read_csv("/content/drive/My Drive/Colab Notebooks/Data/tweet_covid_processing_file.csv", engine='python')
    return data

In [16]:
tweet_df = load_data()
tweet_df.head(5)

,Unnamed: 0,OriginalTweet,Sentiment,label,clean_tweet,tweet_token,tweet_token_filtered,tweet_stemmed,tweet_lemmatized
0,0.0,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,Neutral,0,MeNyrbie Phil Gahan Chrisitv and and,"['MeNyrbie', 'Phil', 'Gahan', 'Chrisitv', 'and...","['MeNyrbie', 'Phil', 'Gahan', 'Chrisitv']",menyrbi phil gahan chrisitv,MeNyrbie Phil Gahan Chrisitv
1,1.0,advice Talk to your neighbours family to excha...,Positive,1,advice Talk to your neighbours family to excha...,"['advice', 'Talk', 'to', 'your', 'neighbours',...","['advice', 'Talk', 'neighbours', 'family', 'ex...",advic talk neighbour famili exchang phone numb...,advice Talk neighbour family exchange phone nu...
2,2.0,Coronavirus Australia: Woolworths to give elde...,Positive,1,Coronavirus Australia Woolworths to give elde...,"['Coronavirus', 'Australia', 'Woolworths', 'to...","['Coronavirus', 'Australia', 'Woolworths', 'gi...",coronaviru australia woolworth give elderli di...,Coronavirus Australia Woolworths give elderly ...
3,3.0,My food stock is not the only one which is emp...,Positive,1,My food stock is not the only one which is emp...,"['My', 'food', 'stock', 'is', 'not', 'the', 'o...","['My', 'food', 'stock', 'one', 'empty', 'PLEAS...",My food stock one empti pleas panic there will...,My food stock one empty PLEASE panic THERE WIL...
4,4.0,"Me, ready to go at supermarket during the #COV...",Extremely Negative,3,Me ready to go at supermarket during the COV...,"['Me', 'ready', 'to', 'go', 'at', 'supermarket...","['Me', 'ready', 'go', 'supermarket', 'COVID', ...",Me readi go supermarket covid outbreak not I p...,Me ready go supermarket COVID outbreak Not I p...


In [17]:
tweet_df.drop('Unnamed: 0',inplace=True,axis = 1)

In [18]:
tweet_df.isnull().sum()

OriginalTweet            0
Sentiment                2
label                    2
clean_tweet              2
tweet_token              2
tweet_token_filtered     2
tweet_stemmed           16
tweet_lemmatized        18
dtype: int64

In [19]:
tweet_df= tweet_df.dropna()
tweet_df.isnull().sum()

OriginalTweet           0
Sentiment               0
label                   0
clean_tweet             0
tweet_token             0
tweet_token_filtered    0
tweet_stemmed           0
tweet_lemmatized        0
dtype: int64

In [20]:
messages= tweet_df.copy()
messages.reset_index()

,index,OriginalTweet,Sentiment,label,clean_tweet,tweet_token,tweet_token_filtered,tweet_stemmed,tweet_lemmatized
0,0,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,Neutral,0,MeNyrbie Phil Gahan Chrisitv and and,"['MeNyrbie', 'Phil', 'Gahan', 'Chrisitv', 'and...","['MeNyrbie', 'Phil', 'Gahan', 'Chrisitv']",menyrbi phil gahan chrisitv,MeNyrbie Phil Gahan Chrisitv
1,1,advice Talk to your neighbours family to excha...,Positive,1,advice Talk to your neighbours family to excha...,"['advice', 'Talk', 'to', 'your', 'neighbours',...","['advice', 'Talk', 'neighbours', 'family', 'ex...",advic talk neighbour famili exchang phone numb...,advice Talk neighbour family exchange phone nu...
2,2,Coronavirus Australia: Woolworths to give elde...,Positive,1,Coronavirus Australia Woolworths to give elde...,"['Coronavirus', 'Australia', 'Woolworths', 'to...","['Coronavirus', 'Australia', 'Woolworths', 'gi...",coronaviru australia woolworth give elderli di...,Coronavirus Australia Woolworths give elderly ...
3,3,My food stock is not the only one which is emp...,Positive,1,My food stock is not the only one which is emp...,"['My', 'food', 'stock', 'is', 'not', 'the', 'o...","['My', 'food', 'stock', 'one', 'empty', 'PLEAS...",My food stock one empti pleas panic there will...,My food stock one empty PLEASE panic THERE WIL...
4,4,"Me, ready to go at supermarket during the #COV...",Extremely Negative,3,Me ready to go at supermarket during the COV...,"['Me', 'ready', 'to', 'go', 'at', 'supermarket...","['Me', 'ready', 'go', 'supermarket', 'COVID', ...",Me readi go supermarket covid outbreak not I p...,Me ready go supermarket COVID outbreak Not I p...
...,...,...,...,...,...,...,...,...,...
41136,41154,Airline pilots offering to stock supermarket s...,Neutral,0,Airline pilots offering to stock supermarket s...,"['Airline', 'pilots', 'offering', 'to', 'stock...","['Airline', 'pilots', 'offering', 'stock', 'su...",airlin pilot offer stock supermarket shelv NZ ...,Airline pilot offering stock supermarket shelf...
41137,41155,Response to complaint not provided citing COVI...,Extremely Negative,3,Response to complaint not provided citing COVI...,"['Response', 'to', 'complaint', 'not', 'provid...","['Response', 'complaint', 'provided', 'citing'...",respons complaint provid cite covid relat dela...,Response complaint provided citing COVID relat...
41138,41156,You know its getting tough when @KameronWilds...,Positive,1,You know it s getting tough when KameronWilds...,"['You', 'know', 'it', 's', 'getting', 'tough',...","['You', 'know', 'getting', 'tough', 'KameronWi...",you know get tough kameronwild ration toilet p...,You know getting tough KameronWilds rationing ...
41139,41157,Is it wrong that the smell of hand sanitizer i...,Neutral,0,Is it wrong that the smell of hand sanitizer i...,"['Is', 'it', 'wrong', 'that', 'the', 'smell', ...","['Is', 'wrong', 'smell', 'hand', 'sanitizer', ...",Is wrong smell hand sanit start turn coronavir...,Is wrong smell hand sanitizer starting turn co...


**A.1 Bag-Of-Words feature matrix - For columns "combine_df['tweet_stemmed']"**

In [21]:
from sklearn.feature_extraction.text import CountVectorizer
bow_vectorizer = CountVectorizer(stop_words="english", analyzer='word', 
                            ngram_range=(1, 1), max_df=1.0, min_df=1, max_features=27000)
bow_vectorizer

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=27000, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words='english',
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [30]:

bow_stem = bow_vectorizer.fit_transform(tweet_df['tweet_stemmed'])
bow_stem

<41141x27000 sparse matrix of type '<class 'numpy.int64'>'
	with 634517 stored elements in Compressed Sparse Row format>

In [26]:
Y_bow_stem = messages['label']

In [27]:
bow_stem.shape,Y_bow_stem.shape

((41141, 27000), (41141,))

**Print the first 10 features**

In [28]:
# Print the first 10 features
print("Every feature:\n{}".format(bow_vectorizer.get_feature_names()))
print("\nEvery 3rd feature:\n{}".format(bow_vectorizer.get_feature_names()[::3]))

Every feature:
['aa', 'aaa', 'aaaaa', 'aaaaakubosan', 'aaaand', 'aaachatterje', 'aaanew', 'aaannnddd', 'aaanortheast', 'aabutan', 'aacopd', 'aacounti', 'aacountygovt', 'aadeshraw', 'aafp', 'aahealth', 'aahh', 'aai', 'aaisp', 'aajeevika', 'aajtak', 'aakash', 'aalonzowatt', 'aalto', 'aaltounivers', 'aalwajih', 'aamaadmi', 'aamaadmiparti', 'aamen', 'aanchalsweet', 'aantal', 'aanthanurdc', 'aanuoluwapoom', 'aao', 'aap', 'aapdelhi', 'aapi', 'aapk', 'aapka', 'aapl', 'aappr', 'aapss', 'aaravnd', 'aaravsingh', 'aardwolfkenya', 'aaron', 'aaronbostermd', 'aaronchown', 'aarondick', 'aaronwinn', 'aarp', 'aarpfraudwatch', 'aartijaidka', 'aartipareek', 'aashith', 'aata', 'aaysharma', 'ab', 'aba', 'ababank', 'abaesq', 'abajila', 'abaloga', 'abandon', 'abangdov', 'abass', 'abat', 'abattoir', 'abbey', 'abbott', 'abbrevi', 'abbyabram', 'abc', 'abcactionnew', 'abcbrisban', 'abccanberra', 'abcdedgar', 'abcnew', 'abcpolit', 'abcworldnew', 'abd', 'abdbozkurt', 'abdijan', 'abdul', 'abdulaziz', 'abdulla', 'ab

**Vocabulary and vocabulary ID**

In [32]:
print("Vocabulary size: {}".format(len(bow_vectorizer.vocabulary_)))
print("Vocabulary content:\n {}".format(bow_vectorizer.vocabulary_))

Vocabulary size: 27000
Vocabulary content:
 {'phil': 14647, 'chrisitv': 5843, 'advic': 452, 'talk': 22311, 'neighbour': 13511, 'famili': 8869, 'exchang': 8725, 'phone': 14660, 'number': 13838, 'creat': 7151, 'contact': 6753, 'list': 12168, 'school': 18500, 'employ': 8470, 'chemist': 5607, 'gp': 9949, 'set': 19032, 'onlin': 14050, 'shop': 19415, 'account': 230, 'poss': 14929, 'adequ': 367, 'suppli': 21891, 'regular': 16604, 'med': 12749, 'order': 14113, 'coronaviru': 6896, 'australia': 2042, 'woolworth': 26178, 'elderli': 8392, 'disabl': 7818, 'dedic': 7504, 'hour': 10649, 'amid': 1254, 'covid': 7069, 'outbreak': 14164, 'food': 9231, 'stock': 21306, 'empti': 8474, 'pleas': 14788, 'panic': 14340, 'everyon': 8691, 'need': 13498, 'stay': 20996, 'calm': 4617, 'safe': 17890, 'franc': 9376, 'confin': 6674, 'confinementot': 6676, 'readi': 16301, 'supermarket': 21842, 'paranoid': 14375, 'seriou': 19006, 'thing': 22924, 'caus': 5105, 'shortag': 19493, 'restezchezv': 16923, 'stayathom': 20999, 'n

In [34]:
from imblearn.combine import SMOTETomek
smk_bow = SMOTETomek(random_state=42)
X_bow,Y_bow=smk_bow.fit_sample(bow_stem ,Y_bow_stem)

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [35]:
from collections import Counter
print('Original dataset shape {}'.format(Counter(Y_bow_stem)))
print('Resampled dataset shape {}'.format(Counter(Y_bow)))

Original dataset shape Counter({'1': 11422, '4': 9917, '0': 7697, '2': 6624, '3': 5481})
Resampled dataset shape Counter({'3': 11357, '2': 11329, '0': 11302, '4': 11262, '1': 11228})


**Spliting dataset**

In [38]:
x_train_bow,x_test_bow,y_train_bow,y_test_bow = train_test_split(X_bow,Y_bow,test_size = 0.30, random_state= True)

**Function of Confusion Matrix**

In [56]:
import matplotlib.pyplot as plt

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    See full source and example: 
    http://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html
    
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

Now it is our turn to make the classifier. We will fit the clean corpus into the machine learning algorithms.

**Random Forest Classifier**

In [43]:
# Random Forest  Classifier
from sklearn.metrics import accuracy_score, confusion_matrix, roc_curve, roc_auc_score
random_forest_biagram = RandomForestClassifier()
random_forest_biagram.fit(x_train_bow,y_train_bow)
y_pred_random_forest_bow = random_forest_biagram.predict(x_test_bow)

predictions_rf = random_forest_biagram.predict(x_test_bow)
print("Random Forest  Classifier")
print(classification_report(y_test_bow, y_pred_random_forest_bow))
print('Accuracy Score:',accuracy_score(y_test_bow, predictions_rf))

# #Confusion matrix
# cm_rf = metrics.confusion_matrix(y_test_bow, y_pred_random_forest_bow)
# plot_confusion_matrix(cm_rf, classes=['T1', 'T2', 'T3', 'T4' ])

Random Forest  Classifier
              precision    recall  f1-score   support

           0       0.54      0.69      0.60      3419
           1       0.50      0.28      0.36      3307
           2       0.53      0.74      0.62      3376
           3       0.63      0.69      0.66      3430
           4       0.52      0.34      0.41      3412

    accuracy                           0.55     16944
   macro avg       0.54      0.55      0.53     16944
weighted avg       0.54      0.55      0.53     16944

Accuracy Score: 0.5485127478753541


**Decision Tree Classifier**

In [45]:

# Decission Tree Classifier
decession_tree_bow = DecisionTreeClassifier()
decession_tree_bow.fit(x_train_bow,y_train_bow)
y_pred_decession_tree_bow = decession_tree_bow.predict(x_test_bow)
predictions_dt = decession_tree_bow.predict(x_test_bow)
print("Decission Tree Classifier")
print(classification_report(y_test_bow, y_pred_decession_tree_bow))
print('Accuracy Score:',accuracy_score(y_test_bow, predictions_dt))

Decission Tree Classifier
              precision    recall  f1-score   support

           0       0.52      0.54      0.53      3419
           1       0.41      0.35      0.38      3307
           2       0.49      0.61      0.54      3376
           3       0.57      0.58      0.58      3430
           4       0.42      0.36      0.39      3412

    accuracy                           0.49     16944
   macro avg       0.48      0.49      0.48     16944
weighted avg       0.48      0.49      0.48     16944

Accuracy Score: 0.4896718602455146


**KNN**

In [47]:
knn_classifier = KNeighborsClassifier()
knn_classifier.fit(x_train_bow,y_train_bow)
y_pred_knn_classifier =knn_classifier.predict(x_test_bow)
predictions_knn = knn_classifier.predict(x_test_bow)
print("KNN  Algo")
print(classification_report(y_test_bow, y_pred_knn_classifier))
print('Accuracy Score:',accuracy_score(y_test_bow, predictions_knn))

KNN  Algo
              precision    recall  f1-score   support

           0       0.34      0.64      0.44      3419
           1       0.65      0.02      0.05      3307
           2       0.43      0.55      0.48      3376
           3       0.44      0.68      0.53      3430
           4       0.50      0.11      0.18      3412

    accuracy                           0.40     16944
   macro avg       0.47      0.40      0.34     16944
weighted avg       0.47      0.40      0.34     16944

Accuracy Score: 0.40321057601510857


**Gradient Boosting**

In [49]:
# Gradient bossting
gradient_bossting_bow  = GradientBoostingClassifier(random_state=3)
gradient_bossting_bow.fit(x_train_bow,y_train_bow)
y_pred_gradientbossting = gradient_bossting_bow.predict(x_test_bow)
#predictions_gb = gradient_bossting_biagram.predict(x_test_biagram)
print("GradientBosting  Algo")
print(classification_report(y_test_bow, y_pred_gradientbossting))
print('Accuracy Score:',accuracy_score(y_test_bow, y_pred_gradientbossting))

GradientBosting  Algo
              precision    recall  f1-score   support

           0       0.42      0.85      0.56      3419
           1       0.44      0.27      0.34      3307
           2       0.58      0.53      0.56      3376
           3       0.65      0.48      0.55      3430
           4       0.46      0.31      0.37      3412

    accuracy                           0.49     16944
   macro avg       0.51      0.49      0.48     16944
weighted avg       0.51      0.49      0.48     16944

Accuracy Score: 0.4909112370160529


**Multinomial Naive Bayes**

In [53]:
from sklearn.naive_bayes import MultinomialNB
classifier=MultinomialNB()
classifier.fit(x_train_bow, y_train_bow)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [61]:
import sklearn.metrics as metrics
predictions = classifier.predict(x_test_bow)
print('Classification Report:\n',classification_report(y_test_bow, predictions))

print('Confusion Matrix:\n',confusion_matrix(y_test_bow, predictions))
print('Accuracy Score:',accuracy_score(y_test_bow, predictions))
print("Precision:",metrics.precision_score(y_test_bow, predictions, average = 'macro'))
print("Recall:",metrics.recall_score(y_test_bow,predictions, average = 'macro'))

Classification Report:
               precision    recall  f1-score   support

           0       0.58      0.56      0.57      3419
           1       0.41      0.42      0.41      3307
           2       0.57      0.62      0.60      3376
           3       0.63      0.60      0.61      3430
           4       0.41      0.39      0.40      3412

    accuracy                           0.52     16944
   macro avg       0.52      0.52      0.52     16944
weighted avg       0.52      0.52      0.52     16944

Confusion Matrix:
 [[1923  505  304  243  444]
 [ 319 1388  828  140  632]
 [ 328  670 2105  100  173]
 [ 366  159  191 2071  643]
 [ 404  665  258  756 1329]]
Accuracy Score: 0.5203021718602455
Precision: 0.5189829977018378
Recall: 0.5197955157674451


**Multinomial Naive Bayes with Hyperparameter**

In [62]:
classifier=MultinomialNB(alpha=0.1)
previous_score=0
for alpha in np.arange(0,1,0.1):
    sub_classifier=MultinomialNB(alpha=alpha)
    sub_classifier.fit(x_train_bow,y_train_bow)
    y_pred=sub_classifier.predict(x_test_bow)
    score = metrics.accuracy_score(y_test_bow, y_pred)
    if score>previous_score:
        classifier=sub_classifier
    print("Alpha: {}, Score : {}".format(alpha,score))

Alpha: 0.0, Score : 0.47574362606232296
Alpha: 0.1, Score : 0.5070231350330501
Alpha: 0.2, Score : 0.5115084985835694
Alpha: 0.30000000000000004, Score : 0.5142823418319169
Alpha: 0.4, Score : 0.5177053824362606
Alpha: 0.5, Score : 0.5183545797922569
Alpha: 0.6000000000000001, Score : 0.5194169027384324
Alpha: 0.7000000000000001, Score : 0.5194169027384324
Alpha: 0.8, Score : 0.5197119924457035
Alpha: 0.9, Score : 0.5207743153918791


**Passive aggressive classifier**

In [63]:
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.metrics import confusion_matrix,accuracy_score
import matplotlib.pyplot as plt 
from sklearn import metrics
import numpy as np
import itertools
linear_clf = PassiveAggressiveClassifier(max_iter=1000, random_state=0,tol=1e-3)
linear_clf.fit(x_train_bow, y_train_bow)
pred = linear_clf.predict(x_test_bow)
score = metrics.accuracy_score(y_test_bow, pred)
print("accuracy:   %0.3f" % score)

accuracy:   0.485


**Logistic Regression classifier**

In [65]:
# Logistic Regression
logis_reg_bow =LogisticRegression()
logis_reg_bow.fit(x_train_bow,y_train_bow)
y_pred_log = logis_reg_bow.predict(x_test_bow)
predictions_lr = logis_reg_bow.predict(x_test_bow)
print("Logistic Reg  Algo")
print(classification_report(y_test_bow, y_pred_log))
print('Accuracy Score:',accuracy_score(y_test_bow, predictions_lr))

Logistic Reg  Algo
              precision    recall  f1-score   support

           0       0.52      0.76      0.62      3419
           1       0.53      0.46      0.49      3307
           2       0.65      0.61      0.63      3376
           3       0.68      0.61      0.64      3430
           4       0.52      0.42      0.46      3412

    accuracy                           0.57     16944
   macro avg       0.58      0.57      0.57     16944
weighted avg       0.58      0.57      0.57     16944

Accuracy Score: 0.5741265344664778


**SVM**

In [66]:
# SVM
import sklearn.metrics as metrics
import itertools
svm_class_bow= svm.SVC(kernel='poly')
svm_class_bow.fit(x_train_bow,y_train_bow)
y_pred_svm = svm_class_bow.predict(x_test_bow)
predictions_svm = svm_class_bow.predict(x_test_bow)
print("SVM  Algo")
print(classification_report(y_test_bow, y_pred_svm))
print('Accuracy Score:',accuracy_score(y_test_bow, predictions_svm))

SVM  Algo
              precision    recall  f1-score   support

           0       0.30      0.90      0.45      3419
           1       0.44      0.25      0.32      3307
           2       0.66      0.30      0.41      3376
           3       0.66      0.38      0.48      3430
           4       0.48      0.18      0.27      3412

    accuracy                           0.40     16944
   macro avg       0.51      0.40      0.38     16944
weighted avg       0.51      0.40      0.38     16944

Accuracy Score: 0.40232530689329554


**A.2 Bag-Of-Words feature matrix - For columns "combine_df[tweet_lemmatized]"**

In [67]:
bow_lemmatized = bow_vectorizer.fit_transform(tweet_df['tweet_lemmatized'])
bow_lemmatized

<41141x27000 sparse matrix of type '<class 'numpy.int64'>'
	with 620701 stored elements in Compressed Sparse Row format>

In [68]:
Y_bow_lemmatize = messages['label']

In [70]:
from imblearn.combine import SMOTETomek
smk_bow = SMOTETomek(random_state=42)
X_lemma,Y_bow_lemmatize=smk_bow.fit_sample(bow_lemmatized ,Y_bow_lemmatize)

**Train Test Split for Lemmatization**

In [72]:
x_train_lemma,x_test_lemma,y_train_lemma,y_test_lemma = train_test_split(X_lemma,Y_bow_lemmatize,test_size = 0.20, random_state= True)

**Machine Learning Model (For Lemmatization: BOW)**

In [80]:
# Random Forest  Classifier
from sklearn.metrics import accuracy_score, confusion_matrix, roc_curve, roc_auc_score
random_forest_lemma = RandomForestClassifier()
random_forest_lemma.fit(x_train_lemma,y_train_lemma)
y_pred_random_forest_lemma = random_forest_lemma.predict(x_test_lemma)
predictions_rf = random_forest_biagram.predict(x_test_bow)
print("Random Forest  Classifier")
print(classification_report(y_test_bow, y_pred_random_forest_bow))
print('Accuracy Score:',accuracy_score(y_test_bow, predictions_rf))


# Decission Tree Classifier
decession_tree_lemma = DecisionTreeClassifier()
decession_tree_lemma.fit(x_train_lemma,y_train_lemma)
y_pred_decession_tree_lemma = decession_tree_lemma.predict(x_test_lemma)
predictions_DT = decession_tree_bow.predict(x_test_lemma)
print("Decission Tree Classifier")
print(classification_report(y_test_lemma, y_pred_decession_tree_lemma))
print('Accuracy Score:',accuracy_score(y_test_lemma, predictions_DT))


# SVM
import sklearn.metrics as metrics
import itertools
svm_class_lemma= svm.SVC(kernel='poly')
svm_class_lemma.fit(x_train_lemma,y_train_lemma)
y_pred_svm = svm_class_bow.predict(x_test_lemma)
predictions_svm = svm_class_bow.predict(x_test_lemma)
print("SVM  Algo")
print(classification_report(y_test_lemma, y_pred_svm))
print('Accuracy Score:',accuracy_score(y_test_lemma, predictions_svm))


#PAC
linear_clf_lemma = PassiveAggressiveClassifier(max_iter=1000, random_state=0,tol=1e-3)
linear_clf_lemma.fit(x_train_lemma, y_train_lemma)
pred_lemma = linear_clf.predict(x_test_lemma)
#score = metrics.accuracy_score(y_test_lemma, pred_lemma)
predictions_pac = svm_class_bow.predict(x_test_lemma)
print(classification_report(y_test_lemma, pred_lemma))
print('Accuracy Score:',accuracy_score(y_test_lemma, predictions_pac))

Random Forest  Classifier
              precision    recall  f1-score   support

           0       0.54      0.69      0.60      3419
           1       0.50      0.28      0.36      3307
           2       0.53      0.74      0.62      3376
           3       0.63      0.69      0.66      3430
           4       0.52      0.34      0.41      3412

    accuracy                           0.55     16944
   macro avg       0.54      0.55      0.53     16944
weighted avg       0.54      0.55      0.53     16944

Accuracy Score: 0.5485127478753541
Decission Tree Classifier
              precision    recall  f1-score   support

           0       0.47      0.53      0.50      2193
           1       0.42      0.35      0.38      2209
           2       0.50      0.59      0.54      2283
           3       0.58      0.61      0.59      2356
           4       0.43      0.34      0.38      2253

    accuracy                           0.49     11294
   macro avg       0.48      0.48      0.48 

In [79]:
# KNN classifier 
knn_classifier_lemma = KNeighborsClassifier()
knn_classifier_lemma.fit(x_train_lemma,y_train_lemma)
y_pred_KNN_classifier =knn_classifier_lemma.predict(x_test_lemma)
predictions_KNN = knn_classifier.predict(x_test_lemma)
print("KNN  Algo")
print(classification_report(y_test_lemma, y_pred_KNN_classifier))
print('Accuracy Score:',accuracy_score(y_test_lemma, predictions_KNN))


# Gradient bossting
gradient_bossting_lemma  = GradientBoostingClassifier(random_state=3)
gradient_bossting_lemma.fit(x_train_lemma,y_train_lemma)
y_pred_gradientbossting = gradient_bossting_lemma.predict(x_test_lemma)
#predictions_gb = gradient_bossting_biagram.predict(x_test_biagram)
print("GradientBosting  Algo")
print(classification_report(y_test_lemma, y_pred_gradientbossting))
print('Accuracy Score:',accuracy_score(y_test_lemma, y_pred_gradientbossting))


# MNB Naive Bayes

MNB_classifier=MultinomialNB()
MNB_classifier.fit(x_train_lemma, y_train_lemma)
predictions_lemma = MNB_classifier.predict(x_test_lemma)
print("MNB naive bayes")
print('Classification Report:\n',classification_report(y_test_lemma, predictions_lemma))
print('Confusion Matrix:\n',confusion_matrix(y_test_lemma, predictions_lemma))
print('Accuracy Score:',accuracy_score(y_test_lemma, predictions_lemma))
print("Precision:",metrics.precision_score(y_test_lemma, predictions_lemma, average = 'macro'))
print("Recall:",metrics.recall_score(y_test_lemma,predictions_lemma, average = 'macro'))


# Logistic Regression
logis_reg_lemma =LogisticRegression()
logis_reg_lemma.fit(x_train_lemma,y_train_lemma)
y_prediction_log = logis_reg_lemma.predict(x_test_lemma)
predictions_lr_lemma = logis_reg_lemma.predict(x_test_lemma)
print("Logistic Reg  Algo")
print(classification_report(y_test_lemma, y_prediction_log))
print('Accuracy Score:',accuracy_score(y_test_lemma, predictions_lr_lemma))


KNN  Algo
              precision    recall  f1-score   support

           0       0.33      0.60      0.42      2193
           1       0.71      0.03      0.06      2209
           2       0.42      0.52      0.47      2283
           3       0.42      0.68      0.52      2356
           4       0.44      0.11      0.17      2253

    accuracy                           0.39     11294
   macro avg       0.46      0.39      0.33     11294
weighted avg       0.46      0.39      0.33     11294

Accuracy Score: 0.21400743757747476
GradientBosting  Algo
              precision    recall  f1-score   support

           0       0.39      0.85      0.53      2193
           1       0.45      0.28      0.34      2209
           2       0.62      0.51      0.56      2283
           3       0.67      0.48      0.56      2356
           4       0.43      0.30      0.35      2253

    accuracy                           0.48     11294
   macro avg       0.51      0.48      0.47     11294
weighted 